In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import config


In [ ]:
API_KEY = config.my_api
print("api key loaded successfully")

In [ ]:
api_url = 'https://data.nasdaq.com/api/v3/datatables/MER/F1.json'
parameters = {
    'api_key': API_KEY,
    'qopts.per_page': 10000
}

json_data = requests.get(api_url,params = parameters).json()

In [ ]:
data = json_data['datatable']['data']
columns = [col['name'] for col in json_data['datatable']['columns']]

df = pd.DataFrame(data, columns=columns)
df['reportdate'] = pd.to_datetime(df['reportdate'],errors='coerce')
df.head()

In [ ]:
df.info()

In [ ]:
necessary_cols = ['reportid','reportdate','reporttype','amount','longname','country','region','indicator','statement']
df = df[necessary_cols]
country_mapping = {
    "USA": "United States",
    "DEU": "Germany",
    "JPN": "Japan",
    "CYM": "Cayman Islands",
    "BHS": "Bahamas",
    "IRL": "Ireland",
    "IND": "India",
    "AUS": "Australia",
    "CAN": "Canada",
    "BRA": "Brazil",
    "IDN": "Indonesia",
    "ISR": "Israel",
    "FIN": "Finland",
    "CHE": "Switzerland",
    "KOR": "South Korea",
    "GBR": "United Kingdom",
    "FRA": "France",
    "CHL": "Chile",
    "BEL": "Belgium",
    "ITA": "Italy",
    "HKG": "Hong Kong",
    "DNK": "Denmark",
    "ESP": "Spain"
}
df['country'] = df['country'].replace(country_mapping)
df

In [ ]:
df.columns = ['report_id','report_date','report_type','amount','company_name','country','region','indicator','statement']
df.head()


In [ ]:
df['country'].value_counts()

In [ ]:
df['indicator'].value_counts()

In [ ]:
filtered_df = df[df['indicator'] == 'EBITDA Margin'].copy()
filtered_df.head()

In [ ]:
filtered_df.describe()

In [ ]:
filtered_df.sort_values(by='amount')

In [ ]:
filtered_df = filtered_df[filtered_df['company_name'] != 'Immutep Ltd']
filtered_df.describe()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(filtered_df['amount'],bins=30,edgecolor='black')
plt.xlabel('EBITDA Margin (%)')
plt.ylabel('Number of Companies')
plt.title('Distribution of EBITDA Margin across Companies')

In [ ]:
plt.figure(figsize=(10,6))
plt.boxplot(filtered_df['amount'])
plt.ylabel('EBITDA Margin(%)')
plt.title('EBITDA Margin Distribution')
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
sns.boxplot(data=filtered_df,x='country', y='amount')
plt.xlabel('Country')
plt.ylabel('EBITDA Margin(%)')
plt.title('EBITDA Margin Distribution by Country')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
time_trend = filtered_df.groupby('report_date')['amount'].mean()
fig , ax = plt.subplots(figsize=(14,6))
ax.plot(time_trend.index, time_trend.values,alpha=0.3,label='Original',linewidth=1)
smoothed = time_trend.rolling(window=3).mean()
ax.plot(smoothed.index,smoothed.values,label='3-period moving average',linewidth=2)

ax.set_xlabel('Report Date')
ax.set_ylabel('Average EBITDA Margin (%)')
ax.set_title('EBITDA Margin Over Time (Smoothed)')
ax.legend()
ax.grid(True,alpha = 0.3)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()